In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126789")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126789
Azure region: southcentralus
Subscription id: f39cb977-6a3a-445b-a26a-b9a791c5fd89
Resource group: aml-quickstarts-126789


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Reference: lesson 5.9 and Microsoft website at:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Jobrunning...............................................................................................................
Running


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)
    

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")



In [28]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps,
                          policy=policy, 
                          primary_metric_name="Accuracy", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 15)

In [29]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config)

#reference: lesson 6.3: running a history widget to show the progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_03953883-04e6-4fac-85a6-8f1812a95bc8
Web View: https://ml.azure.com/experiments/AutoML-project/runs/HD_03953883-04e6-4fac-85a6-8f1812a95bc8?wsid=/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourcegroups/aml-quickstarts-126789/workspaces/quick-starts-ws-126789

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-18T17:41:16.179459][API][INFO]Experiment created<END>\n""<START>[2020-11-18T17:41:16.845886][GENERATOR][INFO]Trying to sample '15' jobs from the hyperparameter space<END>\n""<START>[2020-11-18T17:41:17.124851][GENERATOR][INFO]Successfully sampled '15' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-18T17:41:17.5226742Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe3e489cf60>: Failed to establish a new connection: [Errno 111] Connection refused',)': /history/v1.0/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourceGroups/aml-quickstarts-126789/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-126789/experiments/udacity-project/metrics:query?mergestrategytype=None



Execution Summary
RunId: HD_03953883-04e6-4fac-85a6-8f1812a95bc8
Web View: https://ml.azure.com/experiments/AutoML-project/runs/HD_03953883-04e6-4fac-85a6-8f1812a95bc8?wsid=/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourcegroups/aml-quickstarts-126789/workspaces/quick-starts-ws-126789



In [34]:
# Get your best run and save the model from that run.
#reference lesson 6.3: finding and registering best model

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])



['--C', '0.1', '--max_iter', '400']
Best Run Id:  HD_03953883-04e6-4fac-85a6-8f1812a95bc8_6
Accuracy: 0.9081436519979768
Best C: --C
Best Max_Iter: 0.1


In [35]:
import joblib 

# Save the model
best_run.download_file("/outputs/model.joblib", "Logistic_reg_hd_bankmarketing.joblib")
#model = best_run.register_model(model_name = 'Logistic_reg_hd_bankmarketing', model_path = 'outputs/model.joblib')

In [ ]:
best_run.get_file_names()

In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [38]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [39]:
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [18]:
import pandas as pd

# Because we are doing k-fold cross validation 
# and the validation dataset is not mentioned as a parameter of AutoMLConfig we need to set
#in the assignment, there is no need to split the data into the train and test sets.

# The training_data parameter should contain both training features and a label column 
training_data = pd.concat([x, y], axis = 1)

In [47]:
#Converting Pandas DataFrame to a supported type for the training_data parameter in AutoMLConfig

#Supported types: [azureml.data.tabular_dataset.TabularDataset, 
#azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]"

# Experimental method register_pandas_dataframe does not accept Pandas Series - we cannot convert target variable Y. 
# Therefore, we should use the other method - converting DataFrame to CSV and then to azureml.data.tabular_dataset.TabularDataset type.

# Reference: "Create a dataset from pandas dataframe" section 
# at https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets

from azureml.core import Workspace, Dataset

#creating a folder
#dirname = "./training_data"
os.makedirs('training_data', exist_ok=True)
#try:
#    if not os.path.exists(dirname):
#        os.makedirs(dirname)
#    print("Directory ", dirname, " Created")
#except OSError:
#    print("the directory was not created")

local_path = './training_data/training_data.csv'
training_data.to_csv(local_path)


# upload the local file to a datastore on the cloud
workspace = Workspace(ws.subscription_id, ws.resource_group, ws.name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training_data', target_path='training_data')

# create a dataset referencing the cloud location
training_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ('training_data/training_data.csv'))])




Directory  ./training_data  Created
Uploading an estimated of 1 files
Target already exists. Skipping upload for training_data/training_data.csv
Uploaded 0 files


In [52]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name = "y",
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [54]:
# Submit your automl run

exp = Experiment(workspace=ws, name="AutoML-project")
automl_run = exp.submit(config = automl_config)

# Reference: lesson 6.3: running a history widget to show the progress
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")


Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

In [55]:
# Retrieve and save your best automl model.
# Reference: https://knowledge.udacity.com/questions/357775
automl_best_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) 
print(automl_best_run.get_metrics())
print(automl_run.summary())

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.01,
                                                                                                  min_samples_split=0.056842105263157895,
                                                                                                  min_weight_fraction_leaf=0.0,
     

In [57]:
# Reference: https://knowledge.udacity.com/questions/384083 Mentor: Ahmet Can S

#Save the model
automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')


UserErrorException: UserErrorException:
	Message: File with path /outputs/fitted_automl_model.joblib was not found,
available files include: accuracy_table,automl_driver.py,azureml-logs/55_azureml-execution-tvmps_daad2a4cbbd44f3073af3148c3877c87d7c00003ac9f90759d8ca2866cecc45e_d.txt,azureml-logs/65_job_prep-tvmps_daad2a4cbbd44f3073af3148c3877c87d7c00003ac9f90759d8ca2866cecc45e_d.txt,azureml-logs/70_driver_log.txt,azureml-logs/75_job_post-tvmps_daad2a4cbbd44f3073af3148c3877c87d7c00003ac9f90759d8ca2866cecc45e_d.txt,confusion_matrix,explanation/68bc5289/classes.interpret.json,explanation/68bc5289/eval_data_viz.interpret.json,explanation/68bc5289/expected_values.interpret.json,explanation/68bc5289/features.interpret.json,explanation/68bc5289/global_names/0.interpret.json,explanation/68bc5289/global_rank/0.interpret.json,explanation/68bc5289/global_values/0.interpret.json,explanation/68bc5289/local_importance_values.interpret.json,explanation/68bc5289/per_class_names/0.interpret.json,explan

In [ ]:
cpu_cluster.delete()
